In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split , KFold , cross_val_score
from sklearn.model_selection import GridSearchCV

import xgboost
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

/home/sinho/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv('data/fashion_train.csv')

In [3]:
df_train.head()

,Unnamed: 0,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,7927,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5212,0,0,0,0,0,0,0,0,0,...,23,0,0,0,0,0,0,0,0,0
2,28581,6,0,0,0,0,0,0,0,0,...,141,73,0,0,107,63,0,0,0,0
3,39089,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,52059,4,0,0,0,0,0,0,0,0,...,0,0,0,19,35,7,14,0,0,0


In [4]:
del df_train['Unnamed: 0']

In [5]:
df_train_X = df_train.iloc[:, 1:]
df_train_y = df_train.iloc[:,[0]]

In [6]:
sc = StandardScaler()
df_train_X_std = sc.fit_transform(df_train_X)

In [7]:
pca = PCA(n_components = 100)
X_pca = pca.fit_transform(df_train_X_std)
X_pca.shape

(42000, 100)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_pca,df_train_y , test_size = 0.3 ,random_state= 0, shuffle = True)

In [9]:
def xgboost_Grid(X_train, X_test, y_train, y_test):
        param= { 'n_estimators' : [100],
                      'learning_rate':  [0.01],
                      'max_depth':[6]}
        model_xgb = GridSearchCV(XGBClassifier(), param, scoring="accuracy",n_jobs = -1)
        name = 'XGBoost'
        xgboost_model = model_xgb.fit(X_train, y_train)
        print(name,': ',model_xgb.best_params_)
        print('train score: {}'.format(model_xgb.score(X_train, y_train)))
        print('test score: {}'.format(model_xgb.score(X_test, y_test)))
       
        return model_xgb

In [10]:
%%time
xgboost_Grid(X_train,X_test,y_train,y_test)

XGBoost :  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 100}
train score: 0.8662925170068028
test score: 0.8212698412698413
CPU times: user 40min 10s, sys: 3.84 s, total: 40min 14s
Wall time: 56min 51s


GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100], 'learning_rate': [0.01], 'max_depth': [6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [14]:
def xgboost_Grid(X_train, X_test, y_train, y_test):
        param= { 'n_estimators' : [100,200,300,400,500,600],
                      'learning_rate':  [0.01,0.01,0.1,1,10],
                      'max_depth':[3,4,5,6,7]}
        model_xgb = GridSearchCV(XGBClassifier(), param, scoring="accuracy",n_jobs = 4)
        name = 'XGBoost'
        xgb_model  = model_xgb.fit(X_train, y_train)
        print(name,': ',model_xgb.best_params_)
        print('train score: {}'.format(model_xgb.score(X_train, y_train)))
        print('test score: {}'.format(model_xgb.score(X_test, y_test)))
        
        return model_xgb

In [ ]:
%%time
xgboost_Grid(X_train,X_test,y_train,y_test)